## Conformational Selection using Recursive Feature Elimination

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')

### Load the data

In [2]:
file_name = './df_DkSc_results_COCRYS_DEKOIS_DUD.pkl'
X_merged_dksc = pd.read_pickle(file_name)
# Extract activity column
y_true_merged = X_merged_dksc['activity']
# Drop column from merged_dkksc
X_merged_dksc = X_merged_dksc.drop('activity', axis=1)
X_merged_dksc.shape

(6233, 136)

#### Function and data to perform Scaffold Train-Test splitting

In [3]:
from scaffold_splitter import train_test_scaffold_split

# Compute or load the dataframe containing the Generic Murcko Scaffolds
file = './df_COCRYS_DUD_DEKOIS_Murcko_Scaffolds_SMILES.obj'

df_scff_murcko = pd.read_pickle(file)

In [4]:
# Run or load decorador
from modules.run_or_load_decorator import run_or_load_joblib

### Function to perform RFE with cross Validation

In [5]:
from sklearn.feature_selection import RFECV

import joblib
import os

def _run_RFECV(estimator, X_train, y_train, cv=5, scoring='roc_auc', **kwargs):
    '''Simply runs a RFECV procedure'''
    # Intance and fit the rfe selector
    selector = RFECV(estimator,  cv = cv,
                     scoring = scoring, **kwargs)
    
    selector = selector.fit(X_train, y_train)
    return selector

def get_selected_features_per_step(fitted_selector, X):
    # Get the features ranking
    df_ = pd.DataFrame({'pdb_id': X.columns, 'rfe_ranking': fitted_selector.ranking_})
    # Sort features by ranking
    df_.sort_values('rfe_ranking', inplace = True)
    # Create a list of ranked features from size 1 to n
    list_of_confs_per_k = [ df_.index[:i+1].tolist() for i in range(len(df_))]
    # Create a dataframe indicating which features belong to a k subset
    df_confs_per_k = pd.DataFrame({'list_of_confs_rfe': list_of_confs_per_k})
    return df_confs_per_k

@run_or_load_joblib
def REFCV_wrapper(filename, estimator, X_train, y_train,
                  cv=5, scoring='roc_auc', **kwargs):
    estimator = _run_RFECV(estimator, X_train, y_train, cv=5, scoring='roc_auc', **kwargs)
    return estimator


In [6]:
from sklearn.model_selection import train_test_split
from scaffold_splitter import train_test_scaffold_split
from sklearn.metrics import roc_auc_score

#### Define X and y

In [7]:
# Train and test sets
X = X_merged_dksc
y = y_true_merged

## Random Splitting

#### RANDOM split: Logistic Regression as  estimator

In [15]:
%%time
from sklearn.linear_model import LogisticRegression

# RFECV object to be save 
dataset = 'MERGED'
model_name = 'LogReg'
split = 'random'
filename = f'./ml_models/RFECV_selector_{model_name}_{dataset}_{split}_split.joblib'
# Base Estimator Parameters
hyparams = {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 150}
estimator = LogisticRegression(**hyparams)

#*********************************************************************************
# Do the RANDOM splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

#*********************************************************************************
# Run RFE
selector = REFCV_wrapper(filename, estimator, X_train, y_train, 
                         cv=5, step=1, min_features_to_select=1, n_jobs=4)

selector.grid_scores_

File saved: ./ml_models/RFECV_selector_LogReg_MERGED_random_split.joblib
CPU times: user 5min 6s, sys: 2.67 s, total: 5min 9s
Wall time: 21min 30s


array([0.64410144, 0.67621748, 0.70793292, 0.72448685, 0.74799461,
       0.76208672, 0.77526063, 0.77975007, 0.78200769, 0.78842174,
       0.79363762, 0.79854515, 0.80590277, 0.8049033 , 0.80920361,
       0.81133181, 0.8101341 , 0.80628259, 0.8097683 , 0.80882801,
       0.80900291, 0.81053722, 0.80861397, 0.80674531, 0.80956757,
       0.81411697, 0.81541139, 0.82096529, 0.82123474, 0.8216354 ,
       0.82153494, 0.82250445, 0.82336353, 0.82576949, 0.82457103,
       0.82416632, 0.82592531, 0.82357875, 0.82461829, 0.8253985 ,
       0.82561484, 0.82426681, 0.82328808, 0.82194954, 0.82268943,
       0.82248458, 0.8213962 , 0.82001158, 0.82017183, 0.81908806,
       0.81728936, 0.81765885, 0.81861774, 0.81941739, 0.82079557,
       0.82057033, 0.81976628, 0.81851884, 0.81884363, 0.81808968,
       0.81773997, 0.81896863, 0.81758022, 0.8177351 , 0.81639177,
       0.81672616, 0.81758976, 0.81865861, 0.81839353, 0.81907321,
       0.81932798, 0.81947831, 0.82005779, 0.81934832, 0.81992

#### RANDOM split: Random Forest as estimator 

In [16]:
%%time
from sklearn.ensemble import RandomForestClassifier

# RFECV object to be save 
dataset = 'MERGED'
model_name = 'RandomForest'
split = 'random'
filename = f'./ml_models/RFECV_selector_{model_name}_{dataset}_{split}_split.joblib'
# Base Estimator Parameters
hyparams = {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.02, 
            'min_samples_split': 0.1, 'n_estimators': 300}
estimator = RandomForestClassifier(**hyparams)

#*********************************************************************************
# Do the RANDOM splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

#*********************************************************************************
# Run RFE
selector = REFCV_wrapper(filename, estimator, X_train, y_train, 
                         cv=5, step=1, min_features_to_select=1, n_jobs=4)

selector.grid_scores_

File saved: ./ml_models/RFECV_selector_RandomForest_MERGED_random_split.joblib
CPU times: user 1min 41s, sys: 572 ms, total: 1min 42s
Wall time: 7min 48s


array([0.63783894, 0.69253488, 0.71075666, 0.71859205, 0.71910651,
       0.71139006, 0.70118186, 0.705214  , 0.71606751, 0.7187405 ,
       0.71484486, 0.71031852, 0.71750906, 0.71465752, 0.71750751,
       0.72025849, 0.72323033, 0.72032128, 0.7245774 , 0.72793771,
       0.72916897, 0.73147067, 0.73101616, 0.73397961, 0.73242369,
       0.73520016, 0.73877658, 0.74111666, 0.73978889, 0.73567047,
       0.73556924, 0.74064273, 0.73572686, 0.7399266 , 0.73825127,
       0.73994366, 0.74011018, 0.74475625, 0.7427535 , 0.74406735,
       0.74405675, 0.74618581, 0.74386003, 0.74592332, 0.74739646,
       0.74431694, 0.74516105, 0.74383953, 0.74847931, 0.74826644,
       0.74649786, 0.74410633, 0.7448939 , 0.74544333, 0.74680139,
       0.74465452, 0.74690719, 0.74721581, 0.74555721, 0.74606305,
       0.74869949, 0.74735373, 0.74504224, 0.74444292, 0.74472863,
       0.74876674, 0.7467717 , 0.74341961, 0.74550745, 0.74273305,
       0.74450462, 0.74394996, 0.73983043, 0.7442097 , 0.73978

## Scaffold Splitting

#### Define X and y

In [17]:
# Train and test sets
X = X_merged_dksc
y = y_true_merged
scaffold_series = df_scff_murcko['scff_generic']

#### SCAFFOLD split: Logistic Regression as  estimator

In [21]:
%%time
from sklearn.linear_model import LogisticRegression

# RFECV object to be save 
dataset = 'MERGED'
model_name = 'LogReg'
split = 'scaffold'
filename = f'./ml_models/RFECV_selector_{model_name}_{dataset}_{split}_split.joblib'
# Base Estimator Parameters
hyparams = {'C': 100, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 150}
estimator = LogisticRegression(**hyparams)

#*********************************************************************************
# Do the RANDOM splitting
X_train, X_test, y_train, y_test = train_test_scaffold_split(X, y, 
                                    scaffold_series = scaffold_series,
                                    test_size=0.25, stratify=y)

#*********************************************************************************
# Run RFE
selector = REFCV_wrapper(filename, estimator, X_train, y_train, 
                         cv=5, step=1, min_features_to_select=1, n_jobs=4)
selector.grid_scores_

File saved: ./ml_models/RFECV_selector_LogReg_MERGED_scaffold_split.joblib
CPU times: user 1min 21s, sys: 1.3 s, total: 1min 23s
Wall time: 20min 3s


array([0.66428692, 0.66549177, 0.6993898 , 0.70941584, 0.7192033 ,
       0.74192861, 0.73487809, 0.73388821, 0.74811534, 0.74278356,
       0.75642433, 0.75178342, 0.75878336, 0.75366434, 0.75811005,
       0.76068315, 0.77266108, 0.77099344, 0.77757806, 0.77607828,
       0.77768882, 0.77807311, 0.77899918, 0.77992081, 0.78022081,
       0.78243483, 0.78260477, 0.78473415, 0.78886859, 0.79369802,
       0.79307664, 0.79235934, 0.79586249, 0.79229095, 0.7903834 ,
       0.79031039, 0.79256823, 0.79133774, 0.79514669, 0.79556705,
       0.7958663 , 0.79754487, 0.79856415, 0.79976898, 0.80050597,
       0.79930084, 0.79876064, 0.80068526, 0.80212661, 0.80344725,
       0.80305814, 0.80240535, 0.80176369, 0.80318348, 0.80201042,
       0.80154713, 0.80378967, 0.80419501, 0.8036534 , 0.80326458,
       0.80455309, 0.80510083, 0.80484228, 0.80443207, 0.80516801,
       0.8058352 , 0.80673495, 0.80654478, 0.80878567, 0.8086385 ,
       0.8086391 , 0.80990669, 0.81064812, 0.81032194, 0.81012

#### SCAFFOLD split: Random Forest as estimator 

In [19]:
%%time
from sklearn.ensemble import RandomForestClassifier

# RFECV object to be save 
dataset = 'MERGED'
model_name = 'RandomForest'
split = 'scaffold'
filename = f'./ml_models/RFECV_selector_{model_name}_{dataset}_{split}_split.joblib'
# Base Estimator Parameters
hyparams = {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.02, 
            'min_samples_split': 0.1, 'n_estimators': 400}
estimator = RandomForestClassifier(**hyparams)

#*********************************************************************************
# Do the SCAFFOLD splitting
X_train, X_test, y_train, y_test = train_test_scaffold_split(X, y, 
                                    scaffold_series = scaffold_series,
                                    test_size=0.25, stratify=y)

#*********************************************************************************
# Run RFE
selector = REFCV_wrapper(filename, estimator, X_train, y_train, 
                         cv=5, step=1, min_features_to_select=1, n_jobs=4)
selector.grid_scores_

File loaded: ./ml_models/RFECV_selector_RandomForest_MERGED_scaffold_split.joblib
CPU times: user 373 ms, sys: 16 ms, total: 389 ms
Wall time: 426 ms


array([0.62807247, 0.65434801, 0.68180049, 0.71914251, 0.70185898,
       0.70949886, 0.71749916, 0.7168106 , 0.71777319, 0.73021456,
       0.73064559, 0.73845956, 0.73222367, 0.73472117, 0.73014669,
       0.7303391 , 0.73304695, 0.73231804, 0.73266971, 0.7320845 ,
       0.73012843, 0.73215795, 0.73131385, 0.74233804, 0.74040013,
       0.74264437, 0.73952067, 0.74240573, 0.74052518, 0.7428915 ,
       0.74201735, 0.74108235, 0.73729055, 0.7396859 , 0.74209018,
       0.74045662, 0.73914572, 0.73733552, 0.73797667, 0.7341799 ,
       0.73532397, 0.73809966, 0.74164115, 0.73717583, 0.73732063,
       0.73850945, 0.74305161, 0.74007098, 0.73950101, 0.73785406,
       0.73780903, 0.73992409, 0.7355391 , 0.74218836, 0.74176609,
       0.73612555, 0.7408378 , 0.74455828, 0.73729774, 0.73457117,
       0.73558999, 0.74013687, 0.73814917, 0.74163249, 0.73870498,
       0.73937818, 0.74002351, 0.74002363, 0.73821912, 0.73507202,
       0.73543098, 0.74156108, 0.7385141 , 0.74026242, 0.73733

In [13]:
# Get the features ranking
df_ = pd.DataFrame({'pdb_id': X.columns, 'rfe_ranking': selector.ranking_})
# Sort features by ranking
df_.sort_values('rfe_ranking', inplace = True)
#
list_of_confs_per_k = [ df_.index[:i+1].tolist() for i in range(len(df_))]
df_confs_per_k = pd.DataFrame({'list_of_confs_rfe': list_of_confs_per_k})
df_confs_per_k

,list_of_confs_rfe
0,[135]
1,"[135, 61]"
2,"[135, 61, 60]"
3,"[135, 61, 60, 58]"
4,"[135, 61, 60, 58, 57]"
...,...
131,"[135, 61, 60, 58, 57, 56, 55, 62, 54, 50, 113,..."
132,"[135, 61, 60, 58, 57, 56, 55, 62, 54, 50, 113,..."
133,"[135, 61, 60, 58, 57, 56, 55, 62, 54, 50, 113,..."
134,"[135, 61, 60, 58, 57, 56, 55, 62, 54, 50, 113,..."
